In [ ]:
using Base.Threads, LaTeXStrings, Plots, VantHoffFitting

println(string("Number of available cores for threading: ", nthreads()))

# Introduction

This is a minimal example on how to apply our framework to a set of $\ln(K_{\text{eq}})$-values, measured at different temperatures $T$.  The data should be of the following form:

In [ ]:
example_T = [295.0, 301.0, 310.5] # Kelvin
example_lnK = [-3.0, -1.0, 0.4] # dimensionless
example_ΔlnK = [0.1, 0.07, 0.12] # dimensionless
example_data = hcat([example_T, example_lnK, example_ΔlnK]...)

Note that the code also works without specifying the uncertainties.  The data array would then only contain two columns (in the above example these would be `example_T` and `example_lnK`).  

You can try out the framework on the datasets `dataset_1.dat` and `dataset_2.dat`, which can be found in the subdirectory `./mock_data/`, or try it out on your own data.  Reading in the data goes as follows:

In [ ]:
using DelimitedFiles

data = readdlm("./mock_data/dataset_1.dat") # or use dataset_2

For more details on the theoretical framework, please refer to the associated publication:

> J. T. Bullerjahn and S. Hanson, "Robust analysis of the thermodynamic properties of temperature-sensing molecules", in preparation.  

# Data fitting

The data points are fitted using cubic splines, i.e., piecewise continuous third-order polynomials of the form
\begin{equation*}
S_{i}(s_{i-1} \leq x \leq s_{i}) = \sum_{n=0}^{3} a_{n}^{(i)} (x - s_{i-1})^{n} \, , \qquad i = 1,2,\dots,N,
\end{equation*}
which are joined together in the spline knots $s_{i}$.  The splines satisfy the continuity conditions
\begin{align*}
S_{i}(s_{i}) & = S_{i+1}(s_{i}) \, , \\
S'_{i}(s_{i}) & = S'_{i+1}(s_{i}) \, , \\
S''_{i}(s_{i}) & = S''_{i+1}(s_{i}) \, , 
\end{align*}
as well as some appropriate boundary conditions.  Here, we choose so-called natural boundary conditions, namely
\begin{equation*}
S''_{1}(s_{0}) = S''_{N}(s_{N}) = 0 \, .  
\end{equation*}
The continuity equations and boundary conditions constrain the values of the spline coefficients $a_{n}^{(i)}$, such that only $N+1$ of them can be varied independently.  This is exploited when splines are used to interpolate between a set of points ${x_{i},f(x_{i})}_{i=0}^{N}$, by setting $s_{i} = x_{i}$ $\forall i$ and $a_{0}^{(i)} \equiv S_{i}(s_{i-1}) = f(x_{i-1})$ for $i=1,\dots,N$, as well as $S_{N}(s_{N}) = f(x_{N})$.  

In our fitting procedure, the edge knots $s_{0}$ and $s_{N}$ are held fixed, while the "inner" knots are allowed to vary.  We also vary the values of the splines at the knots.  The best fit minimizes the sum of squared residuals between the data points and splines.  



### Fitting $\ln(K_{\text{eq}})$ as a function of $T$ ("linear model")

Thermodynamics tells us that
\begin{equation*}
\ln (K_{\text{eq}}) = - \frac{\Delta H(T)}{R T} + \frac{\Delta S(T)}{R}
\end{equation*}
must hold, where the enthalpy and entropy differences are defined as
\begin{align*}
\Delta H(T) & = \Delta H(T_{0}) + \int_{T_{0}}^{T} \mathrm{d}T' \Delta C_{p}(T') \, , 
\\
\Delta S(T) & = \Delta S(T_{0}) + \int_{T_{0}}^{T} \mathrm{d}T' \frac{\Delta C_{p}(T')}{T'} \, .  
\end{align*}
The derivative of $\ln(K_{\text{eq}})$ with respect to $T$ is then given by
\begin{equation*}
\frac{\mathrm{d} \ln(K_{\text{eq}})}{\mathrm{d}T} = \frac{\Delta H(T)}{R T^{2}} - \frac{\Delta C_{p}(T)}{R T} + \frac{\Delta C_{p}(T)}{R T} \equiv 
\frac{\Delta H(T)}{R T^{2}} \, ,
\end{equation*}
so the heat capacity difference can be extracted as follows:
\begin{equation*}
\Delta C_{p} = \frac{\mathrm{d} \Delta H(T)}{\mathrm{d}T} = R \frac{\mathrm{d}}{\mathrm{d}T} \bigg[ T^{2} \frac{\mathrm{d} \ln(K_{\text{eq}})}{\mathrm{d}T} \bigg] = 2 R T \frac{\mathrm{d} \ln(K_{\text{eq}})}{\mathrm{d}T} + R T^{2} \frac{\mathrm{d}^{2} \ln(K_{\text{eq}})}{\mathrm{d}T^{2}} \, .  
\end{equation*}

### Fitting $\ln(K_{\text{eq}})$ as a function of $1/T$ ("reciprocal model")

Equivalently, we can also consider the derivative of $\ln(K_{\text{eq}})$ with respect to $1/T$, i.e.,
\begin{equation*}
\frac{\mathrm{d} \ln(K_{\text{eq}})}{\mathrm{d}(1/T)} = \bigg[ \frac{\mathrm{d} (1/T)}{\mathrm{d} T} \bigg]^{-1} \frac{\mathrm{d} \ln(K_{\text{eq}})}{\mathrm{d}T} \equiv 
- \frac{\Delta H(T)}{R} \, ,
\end{equation*}
The heat capacity is then given by
\begin{equation*}
\Delta C_{p} = \frac{\mathrm{d} \Delta H(T)}{\mathrm{d}T} = \frac{\mathrm{d} (1/T)}{\mathrm{d} T} \frac{\mathrm{d} \Delta H(T)}{\mathrm{d}(1/T)} = \frac{R}{T^{2}} \frac{\mathrm{d}^{2} \ln(K_{\text{eq}})}{\mathrm{d}(1/T)^{2}} \, .  
\end{equation*}

# Data analysis

Note that the function `fit_data` uses threading to speed up its calculation.  If you have not done so already, you can run the command `export JULIA_NUM_THREAD=n`, with `n` being the number of available (physical) cores, prior to launching Julia.  Evaluating the top cell of this notebook should print out the number `n` of available cores for threading, if the above-mentioned command was executed properly.  

In [ ]:
model, spline_parameters = fit_data(data)

In [ ]:
# Here, we define a temperature range on which we plot various thermodynamic potentials:
T = collect(floor(data[1,1])-1.0:0.1:ceil(data[end,1])+1.0)
lnK = ln_K.(T, (spline_parameters,), model)

# This function is needed for the plot below to identify whether data contains uncertainties or not.  
function errorbars(data)
    N = size(data,1)
    d = size(data,2)
    if d == 2
        return ones(N)
    else
        return data[:,3]
    end
end

plot(T, lnK, label="splines", legend=:topleft)
pK = plot!(data[:,1], data[:,2], seriestype=:scatter, 
    yerror=errorbars(data), ylim = [ data[1,2]-1, data[end,2]+1 ],
    label="empirical data", 
    xlabel="temperature T [K]", 
    ylabel=string("equilibrium constant ", L"\ln (K_{\mathrm{d}})") )

In [ ]:
dC_p = ΔC_p.(T, (spline_parameters,), model)
dH = ΔH.(T, (spline_parameters,), model)
dS = ΔS.(T, (spline_parameters,), model)
dG = ΔG.(T, (spline_parameters,), model)

p1 = plot(T, dC_p, legend=false, linewidth=2, color=1, 
    xlabel="temperature T [K]", 
    ylabel=string("heat capacity ", L"\Delta C_p"))

p2 = plot(T, dH, legend=false, linewidth=2, color=2, 
    xlabel="temperature T [K]", 
    ylabel=string("enthalpy ", L"\Delta H"))

p3 = plot(T, T .* dS, legend=false, linewidth=2, color=3, 
    xlabel="temperature T [K]", 
    ylabel=string("entropy ", L"T \Delta S"))

plot(data[:,1], -R*data[:,1] .* data[:,2], seriestype=:scatter, 
    yerror=R*data[:,1] .* errorbars(data), label="data", color=12)
p4 = plot!(T, dG, legend=:bottomleft, linewidth=2, color=4, 
    label=string(model, " fit"), 
    xlabel="temperature T [K]", 
    ylabel=string("free energy ", L"\Delta G"))

plot(p1,p2,p3,p4,layout=(2,2))

You can either export the above figures using

In [ ]:
savefig(pK, "./plot1.pdf")
savefig(p1, "./plot2.pdf")
savefig(p2, "./plot3.pdf")
savefig(p3, "./plot4.pdf")
savefig(p4, "./plot5.pdf")

or explicitly write out the data points as follows:

In [ ]:
writedlm("./heat_capacity.dat", hcat([T, dC_p]...))
writedlm("./enthalpy.dat", hcat([T, dH]...))
writedlm("./entropy.dat", hcat([T, dS]...))
writedlm("./free_energy.dat", hcat([T, dG]...))